## Desenvolvimento da 1a Etapa

### Função

In [2]:
import os
def read_directories(directory, img=None):
    # Get a list of filenames in the specified directory
    filenames = []
    for filename in os.listdir(directory):
        if img is not None:
            # If 'img' is provided, filter filenames containing it
            if img in filename:   
                filenames.append(filename)          
        else:
            filenames.append(filename)    
    return filenames

### Diretório das imagens

In [3]:
dir_images = f'work2/images_filtered'
dir_atlas = f'work2/atlas_resampled'
array_images = read_directories(dir_images)
array_atlas = read_directories(dir_atlas)